<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.951 - Tipologia i cicle de vida de les dades</p>
<p style="margin: 0; text-align:right;">2017-2 · Màster universitari en Ciència de dades (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;"><b>Autors:</b> Joan Bonnín i Jose L. Dolz</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# Pràctica 2: Neteja i validació de les dades

## Descripció 
L’objectiu d’aquesta activitat serà el tractament d’un dataset, que pot ser el creat a la pràctica 1 o bé qualsevol dataset lliure disponible a Kaggle (https://www.kaggle.com). Alguns exemples de dataset amb els que podeu treballar són:
<ul>
    <li>Red Wine Quality (https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009).</li>
    <li>Titanic: Machine Learning from Disaster (https://www.kaggle.com/c/titanic).</li>
    <li>Predict Future Sales (https://www.kaggle.com/c/competitive-data-science-predict-future-sales/).</li>
</ul>
Els últims dos exemples corresponen a competicions actives a Kaggle de manera que, opcionalment, podrieu aprofitar el treball realitzat durant la pràctica per entrar en alguna d’aquestes competicions.
Seguint les principals etapes d’un projecte analític, les diferents tasques a realitzar (i <b>justificar</b>) són les següents:


In [ ]:
#NOMÉS USATS PER DEV. ELIMINAR ABANS D'ENTREGAR
from pprint import pprint

# Import all required packages

# Basic maths & data structures
import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats

# Data rendering
from IPython.display import display
import matplotlib.pyplot as plt

In [ ]:
# General constants
PVAL_THRESHOLD = 0.05

### 1. Descripció del dataset. Perquè és important i quina pregunta/problema pretèn respondre?

<font color="#1111aa">
    Per aquesta pràctica hem escollit el conjunt de dades de <a href="https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009" target="_blank">la qualitat del vi negre</a> que, com bé s'explica en la seva pàgina de Kaggle, tracta de les variants del vi portuguès conegut com <a href="http://www.vinhoverde.pt/en/" target="_blank">'Vinho Verde'</a>. Els <b>camps</b> que composen el dataset són tots <b>numérics</b> i són els següents:
    <ul>
        <li><b>fixed acidity</b>: acidesa fixa (<i>g/l</i>). Majoria dels àcids fixes o no volàtils fàcilment.</li>
        <li><b>volatile acidity</b>: acidesa volàtil (<i>g/l</i>). Quantitat d'àcid acètic que en gran quantitat porta al vi a tenir gust a vinagre.</li>
        <li><b>citric acid</b>: àcid cítric (<i>g/l</i>). Quantitat d'aquest àcid (normalment petita) que pot donar sabor i frescura als vins</li>
        <li><b>residual sugar</b>: sucre residual (<i>g/l</i>). Quantitat de sucre que queda després de la fermentació. És estrany trobar vins amb menys d'1 g/l i els que tenem més de 45 g/l es consideren dolços.</li>
        <li><b>chlorides</b>: clorurs (<i>g/l</i>). Representa la quantitat de sal al vi.</li>
        <li><b>free sulfur dioxide</b>: diòxid de sofre lliure o SO2 (<i>mg/l</i>). Parts per milió (ppm) del diòxid que queda lliure un cop es barreja en barrejar-se amb el vi. Aquest diòxid prevé al vi de bactèries i de l'oxidació.</li>
        <li><b>total sulfur dioxide</b>: Diòxid de sofre total (<i>mg/l</i>). La suma (en ppm) de la part lliure i la part fixada al vi de SO2.</li>
        <li><b>density</b>: densitat (<i>g/l</i>). Aquest valor dependrà de la quantitat d'alcohol i sucre en el vi. </li>
        <li><b>pH</b>: descriu com d'àcid o bàsic és un vi en una escala de 0 (molt àcid) a 14 (molt bàsic); La majoria dels vins són entre 3 i 4 en l'escala de pH.</li>
        <li><b>sulphates</b>: sulfats (<i>g/l</i>), que contribueixen al SO2.</li>
        <li><b>alcohol</b>: percentatge de contigut alcohòlic en el vi.</li>
        <li><b>quality</b>: qualitat del vi en una puntuació entre el 0 i el 10.</li>
    </ul>
    Carreguem i fem una ullada al format de les dades:
        

In [ ]:
wine_df = pd.read_csv('winequality-red.csv')
print("El conjunt de dades presenta, com hem vist, {} atributs i està compost per {} registres.".format(wine_df.shape[1], wine_df.shape[0]))
display(wine_df.head())


<font color="#1111aa">
    Com podem veure, tenim els dotze atributs esmentat abans: 11 mesures físico-químiques i la qualitat, que podríem dir que és la classe.
    <p>Aquest conjunt de dades és important perquè ens pot servir per entrenar diferents models d'aprenentatge computacional i poder predir la qualitat d'un vi, mitjançant les dades físicoquímiques obtingudes pels diferents instruments de medició.</p>
    

### 2. Integració i selecció de les dades d’interès a analitzar.

In [ ]:
# Mateixa funció que summary en R. Ho podem fer servir per...
wine_df.describe()


In [ ]:
# Correlació dels atributs
pd.plotting.scatter_matrix(wine_df.iloc[:, :11], figsize=(30,15))
plt.show()

### 3. Neteja de les dades.

#### 3.1. Les dades contenen zeros o elements buits? Com gestionaries aquests casos?

In [ ]:
# Contem els valors nuls
n_nulls = wine_df[wine_df.isnull()].count()

# Contem els zeros
n_zeros = wine_df[wine_df == 0].count()

df_empties = pd.DataFrame()
df_empties["Nombre de Nulls"] = n_nulls
df_empties["Nombre de zeros"] = n_zeros
display(df_empties.T)

<font color="#1111aa">
<p>
    Com podem observar, no tenim cap atribut que presenti valors nuls. Per altra banda, només trobem zeros en l'atribut d'àcid cítric. Concretament, tenim 132 zeros d'un total de 1599 registres, el que representa un 8'26% del total.</p>

<p>
    En aquest cas, no cal que substituim els zeros per cap altra valor. És totalment normal trobar vins negres sense àcid cítric, fet que els hi dona un sabor més anyenc. De fet, com veurem més endavant, el rang de grams per litre de l'àcid cítric en vins negres és molt petit i el zero estaria dintre dels barems. Fins i tot podem trobar vins blancs amb zero gram d'àcid cítric, tot i que en aquests acostuma a haver unes quantitats més altes que als vins negres.</p>
</font>

#### 3.2. Identificació i tractament de valors extrems.

In [ ]:
# Dibueixem un boxplot per cada atribut per veure màxims, 
# mínims, quartils, rang interquartilic i outliers.

df_cols = list(wine_df)
n_rows = 2
n_cols = 6

fig, axes = plt.subplots(n_rows, n_cols, figsize=(30, 15))
for i, row_axes in enumerate(axes):
    for j, ax in enumerate(row_axes):
        idx = i*n_cols + j
        wine_df.boxplot(column=df_cols[idx], ax=axes[i][j], grid=False)
        
plt.show()

Als boxplots s'observa que la majoria de variables presenten una quantitat elevada d'*outliers*, així que val la pena aplicar-hi una anàlisi més extensa per detectar si es tracta de presència de valors erronis o la ralitat de les dades és la que es mostra.

El fet que la majoria d'*outliers* s'agrupin a prop dels bigotis i la densistat baixi segons s'allunyen, ens fa pensar que els valors poden ser correctes, i que simplement la distribució presenta una desviació lateral o una variància molt elevada. Un exemple habitual per mostrar dades que presenten aquesta característica són els salaris.
    
    

### 4. Anàlisi de les dades.
#### 4.1. Selecció dels grups de dades que es volen analitzar/comparar (planificació dels anàlisis a aplicar).

#### 4.2. Comprovació de la normalitat i homogeneïtat de la variància.

In [ ]:
####### NORMALITAT #######

# Mètodes gràfics 
def render_normality_histograms():
    n_rows = 3
    n_cols = 4
    n_bars = 30
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(30, 15))
    for i, row_axes in enumerate(axes):
        for j, ax in enumerate(row_axes):
            idx = i*n_cols + j

            attr = df_cols[idx]
            ax.set_xlabel(attr)      

            data = wine_df[attr].sort_values()
            ax.hist(data, bins=n_bars, normed=True)

            norm = stats.norm.pdf(data, np.mean(data), np.std(data))
            ax.plot(data, norm) 
    plt.show()
        
def plot_qq():
    for attr_key in wine_df:
        attr = wine_df[attr_key]
        stats.probplot(attr, dist="norm", plot=plt)
        plt.show()

# Mètodes numèrics
def test_normality_anderson(render=False):
    normal_attrs = []
    non_normal_attrs = []
    for attr_key in wine_df:
        attr = wine_df[attr_key].values
        res = stats.anderson(attr, dist='norm')
        stat = res.statistic
        threshold = res.critical_values[2] #0.05 significance == PVAL_THRESHOLD
        if stat > threshold:
            normal_attrs.append(attr_key)
        else:
            non_normal_attrs.append(attr_key)
    
    if render:
        print_attrs_distributions(normal_attrs, non_normal_attrs)
        
def test_normality_shapiro(p_value_threshold=PVAL_THRESHOLD, render=False):
    normal_attrs = []
    non_normal_attrs = []
    for attr_key in wine_df:
        attr = wine_df[attr_key].values
        res = stats.shapiro(attr)
        if res[1] > p_value_threshold:
            non_normal_attrs.append(attr_key)
        else:
            normal_attrs.append(attr_key)
            
    if render:
        print_attrs_distributions(normal_attrs, non_normal_attrs)
        
        
def test_normality_dagostino(p_value_threshold=PVAL_THRESHOLD, render=False):
    normal_attrs = []
    non_normal_attrs = []
    for attr_key in wine_df:
        attr = wine_df[attr_key].values
        res = stats.normaltest(attr)
        if res.pvalue > p_value_threshold:
            non_normal_attrs.append(attr_key)
        else:
            normal_attrs.append(attr_key)
            
    if render:
        print_attrs_distributions(normal_attrs, non_normal_attrs)        
    
def print_attrs_distributions(normal, non_normal):
    print("Atributs amb distribució normal: {}".format(normal))
    print("Atributs amb distribució no normal: {}".format(non_normal))
    
# render_normality_histograms()
# plot_qq()

# print("########## Anderson ##########")
# test_normality_anderson(render=True)
# print("##############################", end='\n\n')

# print("########## Shapiro ##########")
# test_normality_shapiro(render=True)
# print("##############################", end='\n\n')

# print("########## D'Agostino ##########")
# test_normality_dagostino(render=True)
# print("##############################", end='\n\n')


In [ ]:
####### HOMOGENEITAT VARIANCIA #######

# Levene és util si la normalitat no està asegurada
def levene_test(threshold=PVAL_THRESHOLD):
    res = stats.levene(*wine_df.as_matrix(), center='median') # Expand matrix to n parameters with '*'
    p = res.pvalue
    return p > threshold
    print(res)
    
    res = stats.levene([1,2,3],[0,1, 71])
    print(res)
    
levene_test()

#### 4.3. Aplicació de proves estadístiques per comparar els grups de dades. En funció de les dades i de l’objectiu de l’estudi, aplicar proves de contrast d’hipòtesis, correlacions, regressions, etc.

In [ ]:
# Mirem les correlacions entre els atributs i la qualitat
# Amb pearson si segueixen distribució normal (que podríem dir que si perque n > 30  pel teorema del límit central)
data = np.ndarray(shape=(len(df_cols), 2))
for i in range(len(df_cols)):
    attr = df_cols[i]
    data[i] = stats.pearsonr(wine_df[attr], wine_df['quality'])
df_pearson = pd.DataFrame(data, index=df_cols, columns=['estimació', 'p-valor'])[:11]
df_pearson['tmp_sort'] = df['estimació'].abs()
df_pearson = df_pearson.sort_values(by='tmp_sort', ascending=False).drop('tmp_sort', axis=1)

# Format output
# pprint(df_pearson['estimació'].values)
df_pearson['estimació'] = ["{0:.4f}".format(el) for el in df_pearson['estimació'].values]
df_pearson['p-valor'] = ["{0:.2e}".format(el) for el in df_pearson['p-valor'].values]
display(df_pearson.T)

### 5. Representació dels resultats a partir de taules i gràfiques.

### 6. Resolució del problema. A partir dels resultats obtinguts, quines són les conclusions? Els resultats permeten respondre al problema?

### 7. Codi: Cal adjuntar el codi, preferiblement en R, amb el que s’ha realitzat la neteja, anàlisi i representació de les dades. Si ho preferiu, també podeu treballar en Python

Tot el codi emprat en aquesta pràctica es pot trobar en les diferens preguntes en les cel·les precedides amb `In [ ]` i es poden executar des de jupyter.